In [1]:
# Training or evaluating model
Training = True # setting to true will run the Trainer.py script, same as if run locally
Testing = True # setting to true will run the Tester.py script, same as if run locally

#Base dir within google drive, change to whatever dir you're using
base_dir = '/content/drive/MyDrive/Colab Notebooks/Honours/'

In [2]:
from google.colab import drive
import os
import shutil
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("torchvision version:", torchvision.__version__)
drive.mount('/content/drive')

PyTorch version: 2.5.1+cu124
CUDA version: 12.4
torchvision version: 0.20.1+cu124
Mounted at /content/drive


In [3]:
# Get github
# Clones pipeline from github repo, uses any settings configured locally within the pipeline
!git clone https://github.com/Stuart1389/SmokeFasterRCNN.git

!pip install torch-pruning
!pip install torchmetrics
!pip install wandb
!pip install super-image
!pip install huggingface-hub==0.25.2


Cloning into 'SmokeFasterRCNN'...
remote: Enumerating objects: 1797, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 1797 (delta 62), reused 63 (delta 27), pack-reused 1688 (from 3)
Receiving objects: 100% (1797/1797), 158.66 MiB | 14.55 MiB/s, done.
Resolving deltas: 100% (1253/1253), done.
Updating files: 100% (101/101), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import wandb
#https://docs.wandb.ai/quickstart/
# to use W&B for other projects, create a project and login
# the pipeline already has metric uploading implemented, once logged in metrics/runs will be uploaded automatically
# make sure logWB is enabled in GetValues.py


In [5]:
pipeline_altered_scripts = '/content/SmokeFasterRCNN/Lib/site-packages'  # folder from github repo with altered site package scripts
target_base_dir = '/usr/local/lib/python3.11/dist-packages' # replacing scripts in this folder


# replacing files
for root, _, files in os.walk(pipeline_altered_scripts):
    relative_path = os.path.relpath(root, pipeline_altered_scripts)
    target_dir = os.path.join(target_base_dir, relative_path)

    os.makedirs(target_dir, exist_ok=True)

    for file in files:
        source_file = os.path.join(root, file)
        target_file = os.path.join(target_dir, file)

        if os.path.exists(target_file):
            os.remove(target_file)
            print(f"Deleted: {target_file}")

        shutil.copy(source_file, target_file)
        print(f"Replaced: {target_file} with {source_file}")

Deleted: /usr/local/lib/python3.11/dist-packages/torchmetrics/detection/mean_ap.py
Replaced: /usr/local/lib/python3.11/dist-packages/torchmetrics/detection/mean_ap.py with /content/SmokeFasterRCNN/Lib/site-packages/torchmetrics/detection/mean_ap.py
Deleted: /usr/local/lib/python3.11/dist-packages/torchvision/models/resnet.py
Replaced: /usr/local/lib/python3.11/dist-packages/torchvision/models/resnet.py with /content/SmokeFasterRCNN/Lib/site-packages/torchvision/models/resnet.py
Deleted: /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py
Replaced: /usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py with /content/SmokeFasterRCNN/Lib/site-packages/torchvision/models/_utils.py
Deleted: /usr/local/lib/python3.11/dist-packages/torchvision/models/feature_extraction.py
Replaced: /usr/local/lib/python3.11/dist-packages/torchvision/models/feature_extraction.py with /content/SmokeFasterRCNN/Lib/site-packages/torchvision/models/feature_extraction.py
Deleted: /usr

In [6]:
import os
# Creating sym links for google drive
# Dataset
Dataset_drive = base_dir + "Dataset"
Dataset_local = "/content/SmokeFasterRCNN/Dataset"
os.symlink(Dataset_drive, Dataset_local)
!ls /content/SmokeFasterRCNN/Dataset


# Dataset h5df
Dataset_drive_h5df = base_dir + "DatasetH5py"
Dataset_local_h5df = "/content/SmokeFasterRCNN/DatasetH5py"
os.symlink(Dataset_drive_h5df, Dataset_local_h5df)
!ls /content/SmokeFasterRCNN/Dataset

# Saved models
savedModels_drive = base_dir + "FasterR-CNN/savedModels"
savedModels_local = "/content/SmokeFasterRCNN/FasterR-CNN/savedModels"
os.symlink(savedModels_drive, savedModels_local)
!ls /content/SmokeFasterRCNN/FasterR-CNN/savedModels


'Large data'  'Medium data'  'Small data'
'Large data'  'Medium data'  'Small data'
003_lr_A100		 hor_flip__A100		     resnet_34_fpnv2_A100
003_lr_L4		 jitter_A100		     resnet_34_fpnv2_L4
004_lr_A100		 large_only_a100	     small_only_a100
004_lr_L4		 medium_only_a100	     test_as_val_a100
101_a100_v1		 no_255_a100		     test_as_val_ctl_a100
101_a100_v2		 normalize_A100		     test_as_val_nv_a100
101_fpnv2_002lr_a100	 no_validation_a100	     test_file_C
101_fpnv2_4tl_a100	 no_validation_L4	     trace_3_no_pin
101_fpnv2_wd_001lr_a100  pinned_non_block_a100	     trace_4_pin
16_006_lr_L4		 place_test		     trace_7_pin_blocking
17_006_lr_A100		 qat_csj_a100		     trace_8_pin_blocking
amp_a100		 qat_csj_a100_am	     transform_csj_a100
BaselineColabA100	 qat_csj_a100_test	     transform_csj_a100_prof
BaselineColabL4		 random_crop_A100	     transform_csj_add_anc_a100
BaselineColabT4		 random_rotate_A100	     transform_csj_add_anc_large_a100
colab_output_L4		 random_scale_A100	     transform_cs

In [7]:
import sys
print(os.path.isdir('/content/SmokeFasterRCNN/Libr'))
sys.path.append('/content/SmokeFasterRCNN/Libr/utils.py')
print(os.listdir('/content/SmokeFasterRCNN/Libr'))

True
['coco_utils.py', 'utils.py', 'coco_eval.py', 'transforms.py']


In [8]:
alteredFasterRcnnDir = base_dir + 'AltModel/detection'
sys.path.append(alteredFasterRcnnDir)

In [9]:
# set dir to run scripts
%cd /content/SmokeFasterRCNN/FasterR-CNN
if (Training):
  # run training script
  !python Trainer.py

if (Testing):
  # run testing script
  !python Tester.py


/content/SmokeFasterRCNN/FasterR-CNN
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
cuda
Default model Mobilenet FpnV3
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100% 74.2M/74.2M [00:00<00:00, 188MB/s]
((32, 64, 128, 256, 512), (32, 64, 128, 256, 512), (32, 64, 128, 256, 512))
Number of parameters: 18871333
108_colab_output_L4
wandb: Currently logged in as: stustu1389gc (stustu1389gc-glasgow-caledonian-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.7
wandb: Run data is saved locally in /content/SmokeF

In [10]:
from google.colab import runtime
# automatically disconect runtime at end
runtime.unassign()
